# Домашнее задание 2
### Выполнила: Кудрявцева Полина Дмитриевна

In [4]:
# импорт библиотек
import pandas as pd
import sqlite3

## Задача 1

**Дан CSV-файл */data/final_transactions.csv*.\
Надо создать таблицу *transaction_bd* со всеми полями, загрузить данные из файла в таблицу и оставить таблицу со структурой:**

(0, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - время транзакций\
(1, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - клиент\
(2, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакций\

In [3]:
# Прочитаем данные из файла и посмотрим на них
transactions_df = pd.read_csv('./data/final_transactions.csv')
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


In [5]:
# Подключимся (и при отсутствии создадим) к базе данных Transactions.db
con = sqlite3.connect("./data/Transactions.db")

In [6]:
# Инициируем курсор
cur = con.cursor()

In [9]:
# Напишем SQL-запрос для создания таблицы transaction_bd со всеми полями из CSV файла и выполним его
create_query = '''
CREATE TABLE transaction_bd(
    TRANSACTION_ID integer NOT NULL PRIMARY KEY,
    TX_DATETIME datetime NOT NULL,
    CUSTOMER_ID integer NOT NULL,
    TERMINAL_ID integer NOT NULL,
    TX_AMOUNT real NOT NULL
);
'''

cur.execute(create_query)
con.commit()

In [12]:
# Запишем данные из датафрейма в таблицу
transactions_df.to_sql("transaction_bd", con, if_exists="replace", index=False)

1048575

## Задача 2 - SQL-запросы к БД
**a) Вывести всех клиентов, у которых сумма транзакций больше 700000 за весь период (сортируя клиентов по возрастанию);**

In [18]:
# Напишем SQL-запрос
query_a = '''
SELECT CUSTOMER_ID, SUM(TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd
GROUP BY CUSTOMER_ID
HAVING TOTAL_TX_AMOUNT > 700000
ORDER BY CUSTOMER_ID ASC
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_a = pd.read_sql(query_a, con)
df_a.head(10)

,CUSTOMER_ID,TOTAL_TX_AMOUNT
0,2249,707478.64
1,3116,721980.69
2,389,753411.90
3,4163,765153.63
4,2891,786115.87


**b) Вывести всех клиентов, у которых сумма транзакций больше 200000 за период 01.01.2023 - 13.01.2023 (сортируя клиентов по возрастанию);** 

In [19]:
# Напишем SQL-запрос
query_b = '''
SELECT CUSTOMER_ID, SUM(TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd
WHERE TX_DATETIME BETWEEN '2023-01-01' AND '2023-01-13'
GROUP BY CUSTOMER_ID
HAVING TOTAL_TX_AMOUNT > 200000
ORDER BY CUSTOMER_ID ASC
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_b = pd.read_sql(query_b, con)
df_b.head(10)

,CUSTOMER_ID,TOTAL_TX_AMOUNT
0,3406,205762.24
1,4252,208090.38
2,1918,209624.38
3,3833,269107.40


**c) Вывести тех клиентов, у которых id начинается с 4 и количество транзакций за весь период более 444;**

In [27]:
# Напишем SQL-запрос
query_c = '''
SELECT CUSTOMER_ID, COUNT(DISTINCT TRANSACTION_ID) AS TRANSACTION_CNT
FROM transaction_bd
WHERE SUBSTRING(CAST(CUSTOMER_ID as varchar), 1, 1) = '4'
GROUP BY CUSTOMER_ID
HAVING TRANSACTION_CNT > 444
ORDER BY TRANSACTION_CNT DESC;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_c = pd.read_sql(query_c, con)
df_c.head(10)

,CUSTOMER_ID,TRANSACTION_CNT
0,4231,451
1,4661,449
2,4539,445


**d) Создать флаг доходности клиентов по логике:**
* Если сумма транзакций не более 50000, тогда вывести 'низкая доходность'.
* Если сумма транзакций больше 50000 и не более 10000 тогда вывести 'средняя доходность'.
* Если сумма транзакций больше 100000 тогда вывести 'высокая доходность'.

In [29]:
# Напишем SQL-запрос
query_d = '''
SELECT TRANSACTION_ID,
       CUSTOMER_ID,
       TX_DATETIME,
       TERMINAL_ID,
       TX_AMOUNT,
       CASE WHEN SUM(TX_AMOUNT) OVER (win) > 100000 THEN 'высокая доходность'
            WHEN SUM(TX_AMOUNT) OVER (win) > 50000 THEN 'средняя доходность'
            ELSE 'низкая доходность'
            END AS YIELD_FLAG
FROM transaction_bd
WINDOW win AS (PARTITION BY CUSTOMER_ID)
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_d = pd.read_sql(query_d, con)
df_d.head(10)

,TRANSACTION_ID,CUSTOMER_ID,TX_DATETIME,TERMINAL_ID,TX_AMOUNT,YIELD_FLAG
0,1758,0,2023-01-01 07:19:05,6076,1200.41,высокая доходность
1,8275,0,2023-01-01 18:00:16,858,751.19,высокая доходность
2,8640,0,2023-01-01 19:02:02,6698,451.79,высокая доходность
3,12169,0,2023-01-02 08:51:06,6569,531.53,высокая доходность
4,15764,0,2023-01-02 14:05:38,7707,614.86,высокая доходность
5,16470,0,2023-01-02 15:13:02,3096,314.24,высокая доходность
6,16810,0,2023-01-02 15:46:51,7502,131.99,высокая доходность
7,18572,0,2023-01-02 20:24:47,3457,504.04,высокая доходность
8,21219,0,2023-01-03 07:41:24,7618,905.83,высокая доходность
9,21852,0,2023-01-03 08:50:09,4726,657.78,высокая доходность


**e) Посмотреть количество клиентов с каждым видом доходности (из пункта 2.d);**

In [30]:
# Напишем SQL-запрос
query_e = f'''
SELECT YIELD_FLAG, COUNT(DISTINCT CUSTOMER_ID) AS CUSTOMER_CNT
FROM ({query_d}) t
GROUP BY YIELD_FLAG;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_e = pd.read_sql(query_e, con)
df_e.head(10)

,YIELD_FLAG,CUSTOMER_CNT
0,высокая доходность,2167
1,низкая доходность,1812
2,средняя доходность,1007


**f) Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).**

In [34]:
# Напишем SQL-запрос
query_f = '''
SELECT date(TX_DATETIME) as TRANSACTION_DT,
       SUM(TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd
GROUP BY TRANSACTION_DT
ORDER BY TRANSACTION_DT ASC
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_f = pd.read_sql(query_f, con)
df_f.head(10)

,TRANSACTION_DT,TOTAL_TX_AMOUNT
0,2023-01-01,4827656.26
1,2023-01-02,4862551.41
2,2023-01-03,5058973.71
3,2023-01-04,4938142.47
4,2023-01-05,5002954.23
5,2023-01-06,5020241.79
6,2023-01-07,4974352.96
7,2023-01-08,5020491.33
8,2023-01-09,5098359.96
9,2023-01-10,5312800.13


In [36]:
# Закроем подключение к БД
con.close()

## Задача 3
**Подготовить дашборд с помощью [Dash](https://dash.plotly.com/) по пункту 2.f, включив туда графики bar и histogram; вставить в конце ноутбука скрин графиков из дашборда.**

![image.png](./data/dashboard_screen.PNG)